In [21]:
from bs4 import BeautifulSoup
import requests
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from datetime import date
from statistics import mean, median
from time import sleep
from datetime import datetime, timedelta
from selenium.webdriver.common.action_chains import ActionChains
from tqdm import tqdm

In [22]:
df = pd.DataFrame({'Артикул':[], 'Количество отзывов': [], 'Рейтинг':[], 'Отзыв': [], 'Оценка отзыва': [], 
                   'Дата отзыва': [], 'Количество фотографий': [],  'id комментатора':[]})
user_df = pd.DataFrame({'id комментатора':[], 'Количество отзывов комментатора':[], 'Заголовок':[], 'Отзыв':[]
                       , 'Оценки по отзывам':[], 'Дата отзыва комментатора':[], 'Количество лет на сайте':[]})
mon = {'01': 'января',
       '02': 'февраля',
       '03': 'марта',
       '04': 'апреля',
       '05': 'мая',
       '06': 'июня',
       '07': 'июля',
       '08': 'августа',
       '09': 'сентября',
       '10': 'октября',
       '11': 'ноября',
       '12': 'декабря'}

In [23]:
url = 'https://www.wildberries.ru/catalog/78317712/feedbacks'

In [24]:
driver = webdriver.Chrome(r'/Users/konstantin_sukhin/Desktop/WB/chromedriver')
driver.get(url)
sleep(7)

<ipython-input-24-4dc98d437d45>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'/Users/konstantin_sukhin/Desktop/WB/chromedriver')


In [25]:
lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight-1000);var lenOfPage=document.body.scrollHeight-1000;return lenOfPage;")
match=False
while(match==False):
    lastCount = lenOfPage
    sleep(3)
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight-1000);var lenOfPage=document.body.scrollHeight-1000;return lenOfPage;")
    if lastCount==lenOfPage:
        match=True

In [26]:
soup=BeautifulSoup(driver.page_source, 'html.parser')
count_feedbacks = soup.find(attrs={"class":{"product-feedbacks__count"}}).text.replace('\xa0', '')
rating = re.findall(r'product-line__rating stars-line star([\d]+)', driver.page_source)[0]
feedbacks = soup.find_all(attrs={"class":{"comments__item feedback j-feedback-slide"}})
for feedback in feedbacks:
    comment = feedback.find_all(attrs={"class":{"feedback__text"}})[0].text
    raiting_comment = re.findall(r'feedback__rating stars-line star([\d]+)', str(feedback))[0]
    date_feedback = feedback.find_all(attrs={"class":{"feedback__date hide-mobile"}})[0].text
    if date_feedback.split(',')[0] == 'Вчера':
        cur_date = (datetime.today().date() - timedelta(days=1)).strftime("%m.%d.%Y")
        date_feedback = str(int(cur_date.split('.')[1]))+' '+mon[cur_date.split('.')[0]] + ','+date_feedback.split(',')[1]
    elif date_feedback.split(',')[0] == 'Сегодня':
        cur_date = (datetime.today().date()).strftime("%m.%d.%Y")
        date_feedback = str(int(cur_date.split('.')[1]))+' '+mon[cur_date.split('.')[0]] + ','+date_feedback.split(',')[1]
    count_photo = 0 if len(feedback.find_all(attrs={"class":{"j-feedback-photo"}})) == 0 else len(feedback.find_all(attrs={"class":{"j-feedback-photo"}}))
    id_user = feedback.find(attrs={"class":{"feedback__header"}}).text
    df = df.append({'Артикул': url.split('/')[4], 'Количество отзывов': count_feedbacks, 'Рейтинг':rating, 'Отзыв': comment, 'Оценка отзыва': raiting_comment, 
                   'Дата отзыва': date_feedback, 'Количество фотографий': count_photo,  'id комментатора':id_user}, ignore_index=True)
    

In [17]:
df

,Артикул,Количество отзывов,Рейтинг,Отзыв,Оценка отзыва,Дата отзыва,Количество фотографий,id комментатора
0,78317712,138,5,так товар топ но я заказал случайно,5,"3 мая, 13:52",0.0,Ахметшин Ринат Ильясович
1,78317712,138,5,"Отличные, ребенок доволен.",5,"29 апреля, 17:33",0.0,Мария
2,78317712,138,5,Отличные наушники),5,"27 апреля, 20:07",0.0,Ление
3,78317712,138,5,"Сыну понравились, посмотрим сколько прослужат....",4,"26 апреля, 13:56",0.0,Анна
4,78317712,138,5,Наушники класс. Пользуюсь 2 дня пока всё ок,5,"23 апреля, 07:32",0.0,Покупатель Wildberries
...,...,...,...,...,...,...,...,...
133,78317712,138,5,"Науниши огонь ! За эту цену, лучше наушиков не...",5,"14 ноября 2022, 16:42",0.0,Дмитрий
134,78317712,138,5,"Приехали вовремя. Работают, звук отличный. За ...",5,"13 ноября 2022, 15:26",1.0,Виктория
135,78317712,138,5,ЭТО ЛУЧШИЕ БЮДЖЕТНЫЕ НАУШНИКИ!,5,"12 ноября 2022, 10:43",2.0,Саид
136,78317712,138,5,Шикарные!!,5,"10 ноября 2022, 19:03",0.0,Ирина


In [18]:
df = df.drop_duplicates()

In [ ]:
for i in tqdm(range(len(df))):
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight-1000);var lenOfPage=document.body.scrollHeight-1000;return lenOfPage;")
    match=False
    while(match==False):
        lastCount = lenOfPage
        sleep(0.3)
        lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight-1000);var lenOfPage=document.body.scrollHeight-1000;return lenOfPage;")  
        try:
            element = driver.find_elements(By.CLASS_NAME,  'feedback__header' )[i]
            action = ActionChains(driver)
            action.move_to_element(to_element=element).perform()
            scroll_vаlue = 200
            scroll_by = f'window.scrollBy(0, {scroll_vаlue});'
            driver.execute_script(scroll_by)  
            soup=BeautifulSoup(driver.page_source, 'html.parser')
            id = soup.find_all(attrs={"class":{"feedback__header"}})[i].text + str(i)
            element.click()
            sleep(0.3)
            break
        except:
            continue
    try:
        while True:
            sleep(0.3)
            lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight-5;return lenOfPage;")
            driver.find_elements(By.CLASS_NAME,  'btn-show-more' )[0].click()            
    except:
        pass
    sleep(5)
    if len(re.findall(r'Профиль не найден', driver.page_source)) == 0:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        while len(soup.find_all(attrs={"class":{"tabs-custom__item active"}}))==0:
            driver.refresh()
            sleep(5)
            soup=BeautifulSoup(driver.page_source, 'html.parser')
        count_feedbacks = soup.find_all(attrs={"class":{"tabs-custom__item active"}})[0].text
        how_long = soup.find_all(attrs={"class":{"person-data__item time"}})[0].text.strip()
        #id = i
        feedbacks = soup.find_all(attrs={"class":{"feedback"}})
        for feedback in feedbacks:
            header = feedback.find(attrs={"class":{"feedback__header j-open-product-popup"}}).text
            rating_feed = re.findall(r'feedback__rating stars-line star([\d]+)', str(feedback))[0]
            feedback_text = feedback.find_all(attrs={"class":{"feedback__text"}})[0].text
            date_feedback = feedback.find_all(attrs={"class":{"feedback__date hide-mobile"}})[0].text
            if date_feedback.split(',')[0] == 'Вчера':
                cur_date = (datetime.today().date() - timedelta(days=1)).strftime("%m.%d.%Y")
                date_feedback = str(int(cur_date.split('.')[1]))+' '+mon[cur_date.split('.')[0]] + ','+date_feedback.split(',')[1]
            elif date_feedback.split(',')[0] == 'Сегодня':
                cur_date = (datetime.today().date()).strftime("%m.%d.%Y")
                date_feedback = str(int(cur_date.split('.')[1]))+' '+mon[cur_date.split('.')[0]] + ','+date_feedback.split(',')[1]
            user_df = user_df.append({'id комментатора':id, 'Количество отзывов комментатора': count_feedbacks, 'Заголовок':header, 'Отзыв':feedback_text, 'Оценки по отзывам':rating_feed, 
                           'Дата отзыва комментатора': date_feedback, 'Количество лет на сайте': how_long}, ignore_index=True)
    driver.get(url)     

  0%|          | 0/138 [00:00<?, ?it/s]

In [ ]:
def find_id_com(feed, user_df = user_df):
    try:
        return user_df.loc[user_df['Отзыв'] == feed]['id комментатора'].iloc[0]
    except:
        return None

In [ ]:
df_try = df.copy()

In [ ]:
df_try['id_for_merge'] = df_try.Отзыв.apply(find_id_com)

In [ ]:
df_total = df_try.merge(user_df, how= 'left', left_on = 'id_for_merge', right_on = 'id комментатора')[['Артикул','Количество отзывов','Рейтинг','Отзыв_x','Оценка отзыва','Дата отзыва','Количество фотографий','id комментатора_x','id_for_merge','Количество отзывов комментатора','Заголовок','Отзыв_y', 'Оценки по отзывам', 'Дата отзыва комментатора', 'Количество лет на сайте']]

In [ ]:
df_total = df_total.loc[df_total.Заголовок.isnull()==False]
df_total

In [ ]:
set(df_total[df_total['Отзыв_y']==df_total['Отзыв_x']]['Заголовок'])

In [ ]:
len(set(df_total['id_for_merge']))

In [ ]:
#df_total.to_excel("78317712_new_wave_1.xlsx")